In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import jieba
import fasttext
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
import os
import glob
from tqdm import tqdm

In [2]:
answer_train=pd.read_csv('answer_train.csv')

In [3]:
encode_dic={}
cate=answer_train.label.unique()
for i in range(len(cate)):
    encode_dic[cate[i]]=i
answer_train['label']=answer_train['label'].map(encode_dic)
filename_label={}
for index,filename,label in answer_train.itertuples():
    filename_label[filename]=label
#

In [4]:
#
decode_dic={}
for key in encode_dic.keys():
    decode_dic[encode_dic[key]]=key
decode_dic

{0: '文化休闲',
 1: '医疗卫生',
 2: '经济管理',
 3: '教育科技',
 4: '城乡建设',
 5: '工业',
 6: '民政社区',
 7: '交通运输',
 8: '生态环境',
 9: '政法监察',
 10: '农业畜牧业',
 11: '文秘行政',
 12: '劳动人事',
 13: '资源能源',
 14: '信息产业',
 15: '旅游服务',
 16: '商业贸易',
 17: '气象水文测绘地震地理',
 18: '财税金融',
 19: '外交外事'}

In [5]:
train_data=pd.read_csv("dataset/train_val_df_127_100.csv")
test_data=pd.read_csv("dataset/test2_df_127_100.csv")
test_data_1=pd.read_csv("dataset/test1_df_127_100.csv")

In [6]:
train_data.shape,test_data.shape,test_data_1.shape

((59985, 2), (25439, 2), (8000, 2))

In [7]:
pseudo_csv=pd.read_csv('submit_a_976.csv')
pseudo_label=[encode_dic[i] for i in pseudo_csv['label'].values.tolist()]
test_data_1['label']=pseudo_label
test_data_1.drop(['name'],axis=1,inplace=True)
train_data_all=pd.concat((train_data,test_data_1)).reset_index(drop=True)

In [8]:
train_data_all

,text,label
0,商务旅行卡审核潍坊市商务旅行卡审核有限公司郝洪乾关于的业务外事侨务办公室山东潍坊集团股份科技...,15
1,小时介绍信息安阳新区上塘上塘安阳新区安阳新区安阳新区安阳新区安阳新区安阳新区安阳新区安阳新区...,8
2,小时评价信息安阳新区上塘上塘安阳新区上塘安阳新区上塘上塘上塘安阳新区上塘上塘上塘上塘安阳新区...,8
3,级旅游厕所初评信息同意万元即墨市旅游建设管理服务平方米企业自筹青岛新建厕所乡村旅游点刘帅康尧...,15
4,级旅游景区名单信息博兴县景区袁收兴国寺韩麻大湖国家湿地公园张敏博物馆滕忠打渔张森林公园张秋博...,15
...,...,...
67980,济南市全市普通高中学校名单山东省济南市中学号高级中学济南市直章丘历城区完全街道商河县学校办事...,3
67981,雁峰区历年地区生产总值,2
67982,辽宁省锦州市北镇市区职业学校一览表,3
67983,民办技工高中和市管民办职业培训高中检查成果信息办学符合要求滨州市职业培训学校技工学校撤销许可...,3


In [10]:
#
#直接利用文件名称即可训练一个fastText模型
train_paths=glob.glob('train/*')
train_paths=[sample.replace('\\','/') for sample in train_paths]
#
save_data='fastTextData_1207/'
if not os.path.exists(save_data):
    os.mkdir(save_data)
train_txt=open(save_data+'train.txt','w',encoding='utf-8')
val_txt=open(save_data+'val.txt','w',encoding='utf-8')
train_val_txt=open(save_data+'train_val.txt','w',encoding='utf-8')
index=0
for _,text,label in tqdm(train_data_all.itertuples()):
    # 结巴分词
    index+=1
    seg_text = jieba.cut(text)
    out_str = " ".join(seg_text)
    if index%100==0:
        val_txt.write('%s __label__%d\n' % (out_str, int(label)))
    else:
        train_txt.write('%s __label__%d\n' % (out_str, int(label)))
    train_val_txt.write('%s __label__%d\n' % (out_str, int(label)))
#
train_txt.close()
val_txt.close()
train_val_txt.close()

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.611 seconds.
Prefix dict has been built successfully.
67985it [05:38, 201.01it/s]


In [5]:
# 2.训练模型
import random
random.seed(2020)
model=fasttext.train_supervised('fastTextData_1207/train.txt',lr=0.5, epoch=15, wordNgrams=2)
#print(model.words)
#print(model.labels)
#验证
texts_test, labels_test = [], []
with open('fastTextData_1207/val.txt', 'r',encoding='utf-8') as f:
    for line in f:
        *text, label = line.strip().split(' ')
        text = ' '.join(text)
        texts_test.append(text)
        labels_test.append(label)
#   
label_encoder = preprocessing.LabelEncoder()
labels_test = label_encoder.fit_transform(labels_test)
predits = list(zip(*(model.predict(texts_test)[0])))[0]
predits = label_encoder.transform(predits)
score = metrics.accuracy_score(predits, labels_test)
print('accuracy_score : %.03f' % score)


accuracy_score : 0.915


In [4]:
save_model='model_save_dir_single/'
if not os.path.exists(save_model):
    os.mkdir(save_model)
save_model=os.path.join(save_model,'fastext_3_'+str(int(score*1000))+'.bin')
model.save_model(save_model)
save_model
# 加载模型
#model = fasttext.load_model(save_model)

'model_save_dir_single/fastext_3_921.bin'

In [6]:
#
test_data=pd.read_csv("test2_df_127_100.csv")
sub_sample=pd.read_csv('submit_example_test2.csv')
sub_filenames=sub_sample['filename'].values
pres=[]
test_paths=glob.glob('test2/*')
test_paths=[sample.replace('\\','/') for sample in test_paths]
#
index=0
tmp_list={}
for i in range(20):
    tmp_list[i]=0
for _,name,text in tqdm(test_data.itertuples()):
    # 结巴分词
    seg_text = jieba.cut(text)
    out_str = " ".join(seg_text)
    most=int(model.predict(out_str)[0][0].split('_')[-1])
    pres.append(most)
#
#解码
pres=[decode_dic[i] for i in pres]
pres

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.438 seconds.
Prefix dict has been built successfully.
25439it [08:51, 47.86it/s]


['城乡建设',
 '政法监察',
 '经济管理',
 '教育科技',
 '工业',
 '生态环境',
 '医疗卫生',
 '文秘行政',
 '生态环境',
 '工业',
 '教育科技',
 '交通运输',
 '文化休闲',
 '城乡建设',
 '交通运输',
 '经济管理',
 '气象水文测绘地震地理',
 '交通运输',
 '经济管理',
 '财税金融',
 '生态环境',
 '城乡建设',
 '教育科技',
 '政法监察',
 '城乡建设',
 '政法监察',
 '工业',
 '工业',
 '教育科技',
 '工业',
 '文化休闲',
 '农业畜牧业',
 '交通运输',
 '文化休闲',
 '财税金融',
 '文化休闲',
 '文秘行政',
 '教育科技',
 '旅游服务',
 '农业畜牧业',
 '文化休闲',
 '医疗卫生',
 '医疗卫生',
 '生态环境',
 '财税金融',
 '工业',
 '城乡建设',
 '工业',
 '教育科技',
 '城乡建设',
 '教育科技',
 '教育科技',
 '工业',
 '医疗卫生',
 '交通运输',
 '生态环境',
 '教育科技',
 '文秘行政',
 '城乡建设',
 '劳动人事',
 '工业',
 '文化休闲',
 '文秘行政',
 '工业',
 '工业',
 '文化休闲',
 '医疗卫生',
 '生态环境',
 '医疗卫生',
 '城乡建设',
 '教育科技',
 '经济管理',
 '工业',
 '医疗卫生',
 '教育科技',
 '教育科技',
 '文秘行政',
 '工业',
 '文化休闲',
 '文化休闲',
 '农业畜牧业',
 '教育科技',
 '医疗卫生',
 '工业',
 '工业',
 '财税金融',
 '医疗卫生',
 '财税金融',
 '教育科技',
 '医疗卫生',
 '文化休闲',
 '财税金融',
 '文化休闲',
 '政法监察',
 '交通运输',
 '教育科技',
 '医疗卫生',
 '城乡建设',
 '经济管理',
 '文化休闲',
 '财税金融',
 '旅游服务',
 '文化休闲',
 '城乡建设',
 '文秘行政',
 '文秘行政',
 '教育科技',
 '政法监察',
 '农业畜牧业',
 '交通运输',
 '财税金融',
 '农业畜牧业',
 '工业',
 '教育

In [7]:
sub_sample['label']=pres
sub_sample.to_csv('submit_12_7_v2.csv',index=False)
sub_sample

,filename,label
0,test2/a7fd17bf2eeb85132a1390dcfd052624.xls,城乡建设
1,test2/辽阳市辽阳县省执法证件信息_.csv,政法监察
2,test2/邯郸市成安县常住人口.csv,经济管理
3,test2/e09b5a9fd3166849eab76945e036d3c5.xls,教育科技
4,test2/ffa49ad5e4d079e2e0b669ab9d76fc7b.xls,工业
...,...,...
25434,test2/云南省普洱市景东彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生
25435,test2/22fdcf3d324a752099cd77bff03d10a6.xls,教育科技
25436,test2/云南省昆明市晋宁县区法人行政许可数据.csv,文秘行政
25437,test2/大洼区区法人行政处罚数据.csv,文秘行政


In [9]:
sub9007=pd.read_csv("9007.csv")

In [10]:
metrics.accuracy_score(sub9007['label'],sub_sample['label'])

0.9280239003105468

In [27]:
#纯标题的用于替换
only_title_df=pd.read_csv("only_title.csv")
title_cls=only_title_df['label'].values
title_name=only_title_df['filename'].values
dic_title={}
for i in range(len(title_cls)):
    dic_title[title_name[i]]=title_cls[i]
#
total_df=sub_sample['filename'].values
result=sub_sample['label'].values
for j in range(len(total_df)):
    if total_df[j] in dic_title:
        result[j]=dic_title[total_df[j]]#用纯标题的结果进行替换
#
submit = {'filename': sub_sample['filename'].values, 'label': result}
sub = pd.DataFrame(submit)
sub.to_csv("submit_12_7_v1_replace_title.csv",index=False)
sub

,filename,label
0,test2/a7fd17bf2eeb85132a1390dcfd052624.xls,城乡建设
1,test2/辽阳市辽阳县省执法证件信息_.csv,政法监察
2,test2/邯郸市成安县常住人口.csv,经济管理
3,test2/e09b5a9fd3166849eab76945e036d3c5.xls,教育科技
4,test2/ffa49ad5e4d079e2e0b669ab9d76fc7b.xls,工业
...,...,...
25434,test2/云南省普洱市景东彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生
25435,test2/22fdcf3d324a752099cd77bff03d10a6.xls,教育科技
25436,test2/云南省昆明市晋宁县区法人行政许可数据.csv,文秘行政
25437,test2/大洼区区法人行政处罚数据.csv,文秘行政


In [29]:
metrics.accuracy_score(sub_sample['label'],sub['label'])

1.0

In [30]:
metrics.accuracy_score(sub8137['label'],sub_sample['label'])

0.8237352097173631

In [31]:
metrics.accuracy_score(sub8137['label'],sub['label'])

0.8237352097173631